In [ ]:
# kaggle 사용법
# If any one still need it. if you are using Linux:
# 1.pip install kaggle
# 2.cd ~/.kaggle
# 3.homepage www.kaggle.com -> Your Account -> Create New API token
# 4.mv ~/Downloads/kaggle.json ./
# 5.chmod 600 ./kaggle.json
#
# This was answered originally here

In [91]:
import gc

import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel , AutoConfig,
    AutoTokenizer , logging
)

In [92]:
import gc ; gc.enable()
from IPython.display import clear_output
import pandas as pd
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')

In [9]:
logging.set_verbosity_error()
logging.set_verbosity_warning()


train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model , config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features  = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)



with torch.no_grad():
    outputs = model(features['input_ids'] , features['attention_mask'])
pooler_output = outputs[1]
logits = nn.Linear(config.hidden_size, 1 ) ( pooler_output) # regression head


print(f"Pooler Output Shape: {pooler_output.detach().numpy().shape } " )
print(f'Logits Shape : {logits.detach().numpy().shape}')

del config, model, tokenizer , features ,   outputs
gc.collect()




Pooler Output Shape: (16, 768) 
Logits Shape : (16, 1)


28617

In [13]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel , AutoConfig,
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()


train_text  = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)


clear_output()

In [14]:
features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

In [15]:

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
last_hidden_state = outputs[0]
attention_mask = features['attention_mask']




In [22]:
attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).shape

torch.Size([16, 256, 768])

In [23]:
input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()

sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded , 1 )
sum_mask = input_mask_expanded.sum(1)
sum_mask = torch.clamp(sum_mask , min=1e-9)
mean_embeddings = sum_embeddings / sum_mask
logits =  nn.Linear(config.hidden_size, 1)(mean_embeddings) # regression head

print(f"Last Hidden State Output Shape:  {last_hidden_state.detach().numpy().shape}")
print(f"Mean Embeddings Output Shape : {mean_embeddings.detach().numpy().shape}")
print(f"Logits Shape : {logits.detach().numpy().shape}")

del config, model, tokenizer, features
gc.collect()

Last Hidden State Output Shape:  (16, 256, 768)
Mean Embeddings Output Shape : (16, 768)
Logits Shape : (16, 1)


112133

In [25]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel , AutoConfig,
AutoTokenizer, logging
)

logging.set_verbosity_warning()
logging.set_verbosity_error()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'


config  = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])

last_hidden_state = outputs[0]
attention_mask = features['attention_mask']

In [26]:
last_hidden_state.size()

torch.Size([16, 256, 768])

In [29]:
#attention_mask.unsqueeze(-1).expand(3)

RuntimeError: expand(torch.LongTensor{[16, 256, 1]}, size=[3]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (3)

In [30]:
input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()

In [31]:
last_hidden_state[input_mask_expanded == 0] = 1e-9 # Set padding tokens to large nagative value
max_embeddings = torch.max(last_hidden_state, 1)[0]

logits = nn.Linear(config.hidden_size, 1)(max_embeddings)

print(f"Last Hidden State Output Shape  : {last_hidden_state.detach().numpy().shape}")
print(f"Max Embeddings Output Shape  : {max_embeddings.detach().numpy().shape}")
print(f"Logits  Shape  : {logits.detach().numpy().shape}")


del config, model,  tokenizer, features
gc.collect()

Last Hidden State Output Shape  : (16, 256, 768)
Max Embeddings Output Shape  : (16, 768)
Logits  Shape  : (16, 1)


24325

In [32]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel , AutoConfig,
AutoTokenizer, logging
)

logging.set_verbosity_warning()
logging.set_verbosity_error()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'


config  = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])

last_hidden_state = outputs[0]

In [ ]:
mean_pooling_embeddings = torch.mean(last_hidden_state, 1)
_, max_pooling_embeddings = torch.max(last_hidden_state, 1)
mean_max_embeddings = torch.cat((mean_pooling_embeddings, max_pooling_embeddings) , 1)

logits = nn.Linear(config.hidden_size*2, 1 )(mean_max_embeddings) # twice the hidden size


print(f"Last Hidden State Output Shape  : {last_hidden_state.detach().numpy().shape}")
print(f"Max Embeddings Output Shape  : {max_embeddings.detach().numpy().shape}")
print(f"Logits  Shape  : {logits.detach().numpy().shape}")





In [48]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel , AutoConfig,
AutoTokenizer, logging
)

logging.set_verbosity_warning()
logging.set_verbosity_error()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'


config  = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])

last_hidden_state = outputs[0]

In [49]:

# first define layers

cnn1 = nn.Conv1d(768, 256, kernel_size=2, padding=1)
cnn2 = nn.Conv1d(256,1 , kernel_size=2, padding=1)

In [50]:
last_hidden_state.shape

torch.Size([16, 256, 768])

In [51]:
last_hidden_state.permute(0,2,1).shape

torch.Size([16, 768, 256])

In [52]:
import torch.nn.functional as F

In [55]:
last_hidden_state = last_hidden_state.permute(0,2,1 )
cnn_embeddings = F.relu(cnn1(last_hidden_state))
cnn_embeddings = cnn2(cnn_embeddings)
logits, _ = torch.max(cnn_embeddings,2)

print(f"Last Hidden State Output Shape: {last_hidden_state.detach().numpy().shape}")
print(f"CNN Embeddings Output Shape: {cnn_embeddings.detach().numpy().shape }")
print(f"Logits Shape : {logits.detach().numpy().shape }")

del config, model, tokenizer, features
gc.collect()

Last Hidden State Output Shape: (16, 768, 256)
CNN Embeddings Output Shape: (16, 1, 258)
Logits Shape : (16, 1)


27004

In [57]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel , AutoConfig,
AutoTokenizer, logging
)

logging.set_verbosity_warning()
logging.set_verbosity_error()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'


config  = AutoConfig.from_pretrained(_pretrained_model)
config.update({'output_hidden_states' : True})
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
all_hidden_states =  torch.stack(outputs[2])

layer_index = 11 # second to last hidden layer
cls_embeddings = all_hidden_states[layer_index+1, :, 0 ] # layer_index + 1 as we have 13 layers (embeddings + num of blocks )


logits = nn.Linear(config.hidden_size , 1 )(cls_embeddings) # regression head



print(f"Last Hidden State Output Shape: {last_hidden_state.detach().numpy().shape}")
print(f"CLS Embeddings Output Shape: {cls_embeddings.detach().numpy().shape }")
print(f"Logits Shape : {logits.detach().numpy().shape }")

del config, model, tokenizer, features
gc.collect()

Last Hidden State Output Shape: (16, 768, 256)
CLS Embeddings Output Shape: (16, 768)
Logits Shape : (16, 1)


24606

In [58]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel , AutoConfig,
AutoTokenizer, logging
)

logging.set_verbosity_warning()
logging.set_verbosity_error()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'


config  = AutoConfig.from_pretrained(_pretrained_model)
config.update({'output_hidden_states' : True})
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
all_hidden_states =  torch.stack(outputs[2])


concatenate_pooling = torch.cat(
    (all_hidden_states[-1], all_hidden_states[-2], all_hidden_states[-3], all_hidden_states[-4]),-1
)

In [59]:
concatenate_pooling = concatenate_pooling[:, 0]

logits = nn.Linear(config.hidden_size* 4,1)(concatenate_pooling) # regression head


print(f" Hidden State Output Shape: {all_hidden_states.detach().numpy().shape}")
print(f"Concatenate Pooling Output Shape: {concatenate_pooling.detach().numpy().shape }")
print(f"Logits Shape : {logits.detach().numpy().shape }")


 Hidden State Output Shape: (13, 16, 256, 768)
Concatenate Pooling Output Shape: (16, 3072)
Logits Shape : (16, 1)


In [60]:
del config , model , tokenizer ,features
gc.collect();

In [61]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel , AutoConfig,
AutoTokenizer, logging
)

logging.set_verbosity_warning()
logging.set_verbosity_error()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'


config  = AutoConfig.from_pretrained(_pretrained_model)
config.update({'output_hidden_states' : True})
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
all_hidden_states =  torch.stack(outputs[2])


concatenate_pooling = torch.cat(
    (all_hidden_states [-1], all_hidden_states[-2], all_hidden_states[-3], all_hidden_states[-4]),-1
)

In [62]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers,  layer_start : int= 4 , layer_weights = None):

        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
            torch.tensor([1] * (num_hidden_layers+1 - layer_start) , dtype=torch.float)


        )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start: , :, :,:]
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor * all_layer_embedding).sum(dim=0) / self.layer_weights.sum()
        return weighted_average

layer_start = 0
pooler = WeightedLayerPooling(
    config.num_hidden_layers,
    layer_start = layer_start, layer_weights=None
)


weighted_pooling_embeddings = pooler(all_hidden_states)
weighted_pooling_embeddings = weighted_pooling_embeddings[: , 0]
logits = nn.Linear(config.hidden_size , 1)(weighted_pooling_embeddings)

print(f" Hidden State Output Shape: {all_hidden_states.detach().numpy().shape}")
print(f"Concatenate Pooling Output Shape: {weighted_pooling_embeddings.detach().numpy().shape }")
print(f"Logits Shape : {logits.detach().numpy().shape }")

del config, model, tokenizer, features
gc.collect()

 Hidden State Output Shape: (13, 16, 256, 768)
Concatenate Pooling Output Shape: (16, 768)
Logits Shape : (16, 1)


61069

In [65]:
pooler.layer_weights.shape

torch.Size([13])

In [66]:
pooler.layer_start

0

In [68]:
pooler

WeightedLayerPooling()

In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import (
    AutoModel, AutoConfig,
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
config.update({'output_hidden_states':True})
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
# outputs[2] == outputs.hidden_states, # 맨 위에 특정 헤드 없이 원시 은닉 상태를 출력하는 베어 RoBERTa 모델 변환,마지막 은닉 상태(출력) 이 아닌 모든 은닉 상태를 가져온다
all_hidden_states = torch.stack(outputs[2])



In [116]:
## lstm network와 intermediate representation of the [CLS] Token 을 연결한다 (x)
#LSTM 네트워크를 사용하여 [CLS] 토큰의 모든 중간 표현을 연결  (o)
class LSTMPooling(nn.Module):
    def __init__(self, num_layers , hidden_size , hiddendim_lstm):
        super(LSTMPooling, self).__init__()
        self.num_hidden_layers = num_layers
        self.hidden_size = hidden_size
        self.hiddendim_lstm = hiddendim_lstm
        self.lstm = nn.LSTM(self.hidden_size , self.hiddendim_lstm, batch_first=True)
        self.dropout = nn.Dropout(0.1)

    def forward(self, all_hidden_states):
        ## forward
        hidden_states =  torch.stack([all_hidden_states[layer_i][:, 0].squeeze() for layer_i in range(1, self.num_hidden_layers+1)],dim=-1)
        hidden_states = hidden_states.view(-1, self.num_hidden_layers, self.hidden_size)


        out, _ = self.lstm(hidden_states, None)
        out = self.dropout(out[:,-1,:])
        return out
hiddendim_lstm = 256
pooler = LSTMPooling(config.num_hidden_layers, config.hidden_size , hiddendim_lstm)
#
lstm_pooling_embeddings = pooler(all_hidden_states)


In [117]:
# n_layer,  batch_size ,max_len, hidden_size
all_hidden_states[0][:,0].shape

torch.Size([16, 768])

In [118]:
# n_layer,  batch_size ,max_len, hidden_size
all_hidden_states[0].shape

torch.Size([16, 256, 768])

In [119]:
# n_layer,  batch_size ,max_len, hidden_size
all_hidden_states[0][:,0].squeeze().shape

torch.Size([16, 768])

In [120]:
# batch_size(하나의 글,처리 단위), hidden_size cls token만 가져오는 코드
hidden_states = torch.stack([all_hidden_states[layer_i][:,0].squeeze() for layer_i in range(1, config.num_hidden_layers+1)],dim=-1)
hidden_states.shape

torch.Size([16, 768, 12])

In [121]:
hidden_states = hidden_states.view(-1, config.num_hidden_layers, config.hidden_size)

In [122]:
hidden_states.shape

torch.Size([16, 12, 768])

In [123]:

print(f" Hidden State Output Shape: {all_hidden_states.detach().numpy().shape}")
print(f"Concatenate Pooling Output Shape: {lstm_pooling_embeddings.detach().numpy().shape }")
print(f"Logits Shape : {logits.detach().numpy().shape }")


del config , model, tokenizer ,  features
gc.collect()

 Hidden State Output Shape: (13, 16, 256, 768)
Concatenate Pooling Output Shape: (16, 256)
Logits Shape : (16, 1)


102762

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import (
    AutoModel, AutoConfig,
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
config.update({'output_hidden_states':True})
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
# outputs[2] == outputs.hidden_states, # 맨 위에 특정 헤드 없이 원시 은닉 상태를 출력하는 베어 RoBERTa 모델 변환,마지막 은닉 상태(출력) 이 아닌 모든 은닉 상태를 가져온다
all_hidden_states = torch.stack(outputs[2])



In [6]:
#
all_hidden_states.shape


torch.Size([13, 16, 256, 768])

In [8]:
# max_len 과 embeddnig 사이즈를 transpose
all_hidden_states.transpose(-2,-1).shape

torch.Size([13, 16, 768, 256])

In [75]:
all_hidden_states[0][:,0].shape

torch.Size([16, 768])

In [77]:
all_hidden_states[0][:,0].squeeze().shape

torch.Size([16, 768])

In [78]:
config.num_hidden_layers

12

In [45]:
# stack을 dim=-1 마지막 차원에 쌓는다
hidden_states = torch.stack([all_hidden_states[layer_i][:,0].squeeze() for layer_i in range(1,config.num_hidden_layers+1)],dim=-1)

In [89]:
torch.stack([all_hidden_states[layer_i][:,0].squeeze() for layer_i in range(1,config.num_hidden_layers+1)],dim=-1).shape

torch.Size([16, 768, 12])

In [90]:
# 모든 레이어를 순회하면서 cls token 값만을 stack하면서 쌓는다

torch.Size([16, 12, 768])

In [48]:
# reshape 한다 tensor를
hidden_states = hidden_states.view(-1, config.num_hidden_layers, config.hidden_size)

In [50]:
hidden_states.shape

torch.Size([16, 12, 768])

In [51]:
# max_len 과 embeddnig 사이즈를 transpose
# query parameter와 matmul
import numpy as np
q_t = np.random.normal(loc=0.0, scale=0.1, size=(1, config.hidden_size))
q = nn.Parameter(torch.from_numpy(q_t)).float()


In [13]:
q.shape

torch.Size([1, 768])

In [52]:
hidden_states.shape

torch.Size([16, 12, 768])

In [53]:
hidden_states.transpose(-2,-1).shape

torch.Size([16, 768, 12])

In [54]:
#query와 matmul 하여 하나의 값으로 만들도록 matmul
torch.matmul(q,hidden_states.transpose(-2,-1)).shape

torch.Size([16, 1, 12])

In [55]:
v = torch.matmul(q,hidden_states.transpose(-2,-1)).squeeze(1)

In [56]:
v.shape

torch.Size([16, 12])

In [57]:
# 문장 max_len 에서 마지막 차원에 softmax화
v_ = F.softmax(v,-1)

In [58]:
v_.shape

torch.Size([16, 12])

In [59]:
torch.matmul(v_.unsqueeze(1), hidden_states).shape

torch.Size([16, 1, 768])

In [60]:
torch.matmul(v_.unsqueeze(1), hidden_states).shape

torch.Size([16, 1, 768])

In [61]:
v_temp = torch.matmul(v_.unsqueeze(1), hidden_states).transpose(-2,-1)

In [62]:
v_temp.shape

torch.Size([16, 768, 1])

In [63]:
hiddendim_fc = 128

In [64]:
w_ht = np.random.normal(loc=0.0, scale=0.1, size=(config.hidden_size,hiddendim_fc))

In [65]:
w_h = nn.Parameter(torch.from_numpy(w_ht)).float()

In [66]:
w_h.shape

torch.Size([768, 128])

In [67]:
torch.matmul(w_h.transpose(1,0), v_temp
).shape

torch.Size([16, 128, 1])

In [68]:
final_v = torch.matmul(w_h.transpose(1,0), v_temp
).squeeze(2)

In [69]:
final_v.shape

torch.Size([16, 128])

In [125]:
train_text

['When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThese rugs were for prizes, one for the girls and one for the boys. And this was the game.\nThe girls were gathered at one end of the room and the boys at the other, and one end was called the North Pole, and the other the South Pole. Each player was given a small flag which they were to plant on reaching the Pole.\nThis would have been an easy matter, but each traveller was obliged to wear snowshoes.'

In [ ]:
import numpy as np

In [ ]:
class AttentionPooling(nn.Module):
    def __init__(self, num_layers, hidden_size , hiddendim_fc):
        super(AttentionPooling, self).__init__()
        self.num_hidden_layers = num_layers
        self.hidden_size  = hidden_size
        self.hiddendim_fc =  hiddendim_fc
        self.dropout = nn.Dropout(0.1)

        q_t =  np.random.normal(loc=0.0 , scale=0.1 , size=(1, self.hidden_size))
        self.q = nn.Parameter(torch.from_numpy(q_t)).float()
        w_ht = np.random.normal(loc=0.0 , scale=0.1 , size=(self.hidden_size , self.hiddendim_fc))

        self.w_h = nn.Parameter(torch.from_numpy(w_ht)).float()

    def forward(self, all_hidden_states):
        # cls token을 뽑는 과정
        hidden_states =  torch.stack([all_hidden_states[layer_i][: , 0].squeeze() for layer_i in range(1, self.num_hidden_layers)])
        hidden_states = hidden_states.view(-1, self.num_hidden_layers, self.hidden_size)

        out = self.attention(hidden_states)
        out = self.dropout(out )
        return out
    # o = W_h^T*softmax(q*h^T_cls)h_cls 를 구현한 것이다.
    #https://www.kaggle.com/code/rhtsingh/utilizing-transformer-representations-efficiently
    def attention(self, h ):
        # attention operation은 학습할 수 있다.
        # the contribution of each h^i_cls   : (h의 각 i번째 cls 의 분포를, 그래서 attention, 주목해야할 부분을 찾는다 캬.. 실제적으로 생각하면서
        # 코드 보고 생각하고 수식 실제적으로 생각하니 이해해냈다)
        v = torch.matmul(self.q , h.transpose(-2,-1)).squeeze(1)
        v = F.softmax(v, -1)
        v_temp = torch.matmul(v.unsqueeze(1), h).transpose(-2,-1 )
        v, = torch.matmul(self.w_h.transpose(1,0) , v_temp).squeeze(2)
        return v


hiddendim_fc = 128
pooler = AttentionPooling(config.num_hidden_layers, config.hidden_size, hiddendim_fc)
attention_pooling_embeddings = pooler(all_hidden_states)
logtis = nn.Linear(hiddendim_fc, 1)(attention_pooling_embeddings)  # regression head

# detach : copy
print(f" Hidden State Output Shape: {all_hidden_states.detach().numpy().shape}")
print(f"Concatenate Pooling Output Shape: {attention_pooling_embeddings.detach().numpy().shape }")
print(f"Logits Shape : {logits.detach().numpy().shape }")


del config , model , tokenizer,  features
gc.collect()


In [127]:
import numpy as np

In [ ]:

q_t = np.random.normal(loc=0.0 , scale=0.1 , size=(1, config.hidden_size))
 # 말 그대로 파라미터만 가지고 있다
 #%
# %

In [136]:
q = nn.Parameter(torch.from_numpy(q_t)).float()

In [137]:
hidden_states =  torch.stack([all_hidden_states[layer_i][:, 0].squeeze() for layer_i in range(1, config.num_hidden_layers+1)], dim=-1)

In [139]:
hidden_states.shape

torch.Size([16, 768, 12])

In [140]:
hidden_states = hidden_states.view(-1, config.num_hidden_layers , config.hidden_size)

In [142]:
hidden_states.shape

torch.Size([16, 12, 768])

In [146]:
hidden_states.transpose(-2,-1).shape

torch.Size([16, 768, 12])

In [153]:
q.shape

torch.Size([1, 768])

In [152]:
# matmul 의 경우 두번째 dimension 과 서로 같아야  계산 가능 matmul은 주어진 차원에 따라 계산 구현이 다름
torch.matmul(q, hidden_states.transpose(-2,-1)).squeeze(1).shape

torch.Size([16, 12])

torch.Size([1, 768])

0.112166

In [131]:
q_t

array([[ 1.50340343e-01,  4.58478509e-02, -7.53045705e-02,
         8.09586386e-03,  9.93280388e-02,  1.80328811e-02,
        -5.54247503e-02,  8.95468500e-02,  2.10247239e-02,
        -2.66704411e-02, -1.18155986e-02, -4.64011779e-02,
        -1.43496801e-01,  1.11578032e-01,  4.67111154e-03,
        -8.46342745e-02, -1.12850845e-01,  1.48897500e-01,
         6.00778453e-02,  1.65535395e-02,  4.46960835e-02,
         1.39637997e-01,  2.09386843e-01,  2.33615016e-01,
         1.85463817e-01,  1.51698747e-01, -4.09955507e-02,
        -1.34423575e-02, -1.14128348e-01, -2.64406672e-02,
        -3.27826966e-02, -1.33009834e-01,  3.78011421e-02,
        -1.19426698e-01, -5.00464016e-02, -7.45591960e-03,
         6.06261634e-02,  2.09348093e-01, -1.33766525e-01,
         2.42471421e-01,  1.52789052e-01,  1.24875000e-01,
         4.75891504e-02, -7.68287647e-02, -8.63998072e-02,
         7.63915132e-02, -5.87534013e-02,  3.02108096e-02,
         1.46235568e-01,  1.22103722e-01, -1.62984035e-0